In [94]:
## imports
import csv
import random as rand
import copy

In [102]:
class Job():
    def __init__(self, initialPosition, processingTime, earlinessPenalty, tardinessPenalty, piAi, piBi, deltaAiBi, penaltyTime):
        self.initialPosition = initialPosition
        self.processingTime = processingTime
        self.earlinessPenalty = earlinessPenalty
        self.tardinessPenalty = tardinessPenalty
        self.piAi = piAi
        self.piBi = piBi
        self.deltaAiBi = deltaAiBi
        self.penaltyTime = penaltyTime
        
    def __str__(self):
        return self.initialPosition

In [103]:
def readJobsFromFile():    
    jobs = []

    spamReader = csv.reader(open('Jobs.csv', newline=''), delimiter=',', quotechar='|')
    index = 1
    for row in spamReader:
        job = Job(index,row[0], row[1], row[2],0,0,0,0)
        jobs.append(job)
        index += 1

    return jobs

In [104]:
def ordeanandoDeltaAiBi(jobs):
    
    vetorOrdenado= []

    for job in jobs:
        if int(job.tardinessPenalty) >= int(job.earlinessPenalty):
            job.deltaAiBi = int(job.tardinessPenalty) - int(job.earlinessPenalty)
        else:
            job.deltaAiBi = int(job.earlinessPenalty) - int(job.tardinessPenalty)         
    vetorOrdenado =  sorted(jobs, key=lambda k: k.deltaAiBi,reverse=True )
        
    return vetorOrdenado

In [105]:
def calculatePiAiBi(jobs):
    for job in jobs:
        job.piAi = int(job.processingTime) / int(job.earlinessPenalty) 
        job.piBi = int(job.processingTime) / int(job.tardinessPenalty) 
    return jobs

In [106]:
def calculateDueDate(jobs, percentage):
    sum = 0
    for job in jobs:
        sum += int(job.processingTime);
    
    return sum * percentage;

In [107]:
def calculateJobsBeforeDueDate(vetorOrdenado, dueDateLimit):
    
    beforeDueDate = []
        
    sumOfProcessingTime = 0
    for job in vetorOrdenado:
        if sumOfProcessingTime + int(job.processingTime) >= dueDateLimit:
            break
        if (int (job.tardinessPenalty)- int(job.earlinessPenalty))>=0:
            sumOfProcessingTime += int(job.processingTime)
            beforeDueDate.append(job)
             
        
    beforeDueDate = sorted(beforeDueDate, key=lambda k: k.piAi,reverse=True )          
      
    return beforeDueDate;

In [108]:
def calculateJobsAfterDueDate(vetorOrdenado, beforeDueDate):
    
    afterDueDate = []
        
    for job in vetorOrdenado:
        if job not in beforeDueDate:
            afterDueDate.append(job)
            
    afterDueDate = sorted(afterDueDate, key=lambda k: k.piAi )
    
    return afterDueDate;

In [109]:
def merge(beforeDueDate, afterDueDate):
    
    result = beforeDueDate + afterDueDate

    return result

In [110]:
def getSumOfPenalties(resultAfterFO):
    sumOfPenalties = 0 

    for job in resultAfterFO:
        sumOfPenalties += int(job.penaltyTime)
        
    return sumOfPenalties;

In [111]:
def getProcessingTime(resultAfterFO):
    processingTime = 0 

    for job in resultAfterFO:
        processingTime += int(job.processingTime)
        
    return processingTime;

In [112]:
def objectiveFunction(result, dueDateLimit, beforeDueDate):
    completion_Time = int(dueDateLimit - getProcessingTime(beforeDueDate)) 

    resultAfterFO = [] 
    for job in result:
        completion_Time= completion_Time + int(job.processingTime)
        if(completion_Time < dueDateLimit):
            job.penaltyTime = (int(dueDateLimit) - completion_Time) * int(job.earlinessPenalty)
            resultAfterFO.append(job)
        elif(completion_Time == dueDateLimit):
            job.penaltyTime = (int(dueDateLimit) -completion_Time)
            resultAfterFO.append(job)
        elif(completion_Time > dueDateLimit):
            job.penaltyTime = (completion_Time - int(dueDateLimit)) * int(job.tardinessPenalty)
            resultAfterFO.append(job)
    return resultAfterFO

In [113]:
def constructiveHeuristic(jobs, percentage):
    # passo 02 - calcular o dueDate
    dueDateLimit = calculateDueDate(jobs,percentage);
    
    # passo 03 - calcular o PiAi e o PiBi
    ordenaPiAiBi = calculatePiAiBi(jobs)      
    # passo 04 - Ordenar    
       
    #ordenando o vetovocr pela diferenca bi-ai
    vetorOrdenadoDelta= ordeanandoDeltaAiBi(jobs)
    
   
    # passo 05 - calcular o beforeDueDate
    beforeDueDateFinal  = calculateJobsBeforeDueDate(vetorOrdenadoDelta, dueDateLimit)
    afterDueDateFinal = calculateJobsAfterDueDate(vetorOrdenadoDelta, beforeDueDateFinal)
    # passo 06 - Merge    
    result = merge(beforeDueDateFinal,  afterDueDateFinal)
    # passo 07 - Calcular a função objetivo
    resultsAfterFO = objectiveFunction(result, dueDateLimit, beforeDueDateFinal)
    # passo 08 = mostrar resultados
    sumOfPenalties = getSumOfPenalties(resultsAfterFO)
    sumOfProcessingTime = getProcessingTime(resultsAfterFO)
    
    return resultsAfterFO
    #print(dueDateLimit)
    #print("\n")
    #print("before")
    #for a in beforeDueDateFinal:
        #print (a['initial_position'])
    #print("\n")
    #print("after")    
    #for a in afterDueDateFinal:
        #print (a['initial_position'])
    #print(result)
    #print(sumOfPenalties)
    #print("sum of processing time: ", sumOfProcessingTime)
    #print("final result: ", sumOfPenalties + sumOfProcessingTime)
    

In [114]:
jobs = readJobsFromFile()
k = 1
for i in range(0,100,10):
    #print("----------------- k =", k, "--------------------------" )
    constructiveHeuristic(jobs[i:i+10], 0.6)
    k +=1
    #print("--------------------------------------------------\n")
    
    

In [115]:
def getBeforeDueDate(listToSplit, dueDate):
    beforeDueDate = []
    completion_time = 0
    for j in listToSplit:
        if completion_time + int(j.processingTime) < dueDate:
            beforeDueDate.append(j)
    return beforeDueDate

In [116]:
#pos = posicao que sera trocada
#sol = solucao que sera trocada
#n_elmentos= n_elementos da solucao

def disturbing (listToDisturb):
    toReturn = copy.deepcopy(listToDisturb)
    rand.shuffle(toReturn)
    return toReturn
    

In [125]:
def heuristicaMelhoria (jobs, percentage, depth):
    

    listaAfterFO = constructiveHeuristic(jobs, percentage)
    melhor_solucao = listaAfterFO.copy()
    for k in range(0,depth):
        toDisturb = copy.deepcopy(melhor_solucao)
        vizinho = disturbing(toDisturb)
        #print("\n -------- melhor solucao antes da FO --------\n")
        #for m in melhor_solucao:
        #    print(m.initialPosition, end=" ")
        #print("soma da melhor solução antes da FO: ", getSumOfPenalties(melhor_solucao))

        dueDateMelhoria = calculateDueDate(vizinho.copy(),percentage)
        beforeDueDateMelhoria = getBeforeDueDate (vizinho.copy(),dueDateMelhoria)
        results = objectiveFunction (vizinho.copy(), dueDateMelhoria,beforeDueDateMelhoria)

        #print("\n -------- melhor solucao --------\n")
        #for m in melhor_solucao:
        #    print(m.initialPosition, end=" ")
        #print("soma da melhor solução: ", getSumOfPenalties(melhor_solucao))
        
        #print("se : ",getSumOfPenalties(results), " < ", getSumOfPenalties(melhor_solucao) )
        if getSumOfPenalties (results) < getSumOfPenalties(melhor_solucao) :
        #    print("entreii mano")
            melhor_solucao = results.copy()
    for i in melhor_solucao:
        print(i.initialPosition, end = ' ')
    print("soma de penalidade: ", getSumOfPenalties(results))
    return melhor_solucao

In [129]:
jobsMelhoria = readJobsFromFile()
k = 1
for i in range(0,100,10):
    #print("----------------- k =", k, "--------------------------" )
    heuristicaMelhoria(jobsMelhoria[i:i+10], 0.2, 10000 )
    k +=1
    #print("--------------------------------------------------\n")

1 4 2 7 3 9 10 6 5 8 soma de penalidade:  2272
15 17 19 11 18 16 14 12 13 20 soma de penalidade:  4028
27 24 22 28 21 29 26 23 25 30 soma de penalidade:  4152
36 40 31 32 33 35 34 37 39 38 soma de penalidade:  2426
45 49 42 47 50 44 43 46 41 48 soma de penalidade:  1506
58 57 52 54 60 55 56 53 51 59 soma de penalidade:  1767
68 63 66 61 65 70 64 67 69 62 soma de penalidade:  2979
80 77 74 71 79 78 76 75 72 73 soma de penalidade:  1850
84 81 87 88 85 90 89 82 86 83 soma de penalidade:  1908
100 99 91 98 93 95 92 97 96 94 soma de penalidade:  2426
